# 1. pdf 페이지 단위로 청킹

In [7]:
import os
import pymupdf  # fitz

def split_pdf(filepath, output_dir="split_pdf"):
    """
    PDF를 페이지 단위로 분리하여 output_dir 폴더에 저장
    파일명은 001.pdf, 002.pdf ... 형식
    """
    # 출력 폴더 생성
    os.makedirs(output_dir, exist_ok=True)

    input_pdf = pymupdf.open(filepath)
    num_pages = len(input_pdf)
    print(f"총 페이지 수: {num_pages}")

    ret = []

    for page_num in range(num_pages):
        # 저장 경로 지정
        output_file = os.path.join(output_dir, f"{page_num+1:03d}.pdf")

        with pymupdf.open() as output_pdf:
            output_pdf.insert_pdf(input_pdf, from_page=page_num, to_page=page_num)
            output_pdf.save(output_file)
            ret.append(output_file)

        print(f"저장 완료: {output_file}")

    input_pdf.close()
    return ret


# ===== 사용 예시 =====
if __name__ == "__main__":
    filepath = "무역관정산교육_v6_0923.pdf"
    result_files = split_pdf(filepath, output_dir="split_pdf")

    print("총 분리된 파일 수:", len(result_files))

총 페이지 수: 49
저장 완료: split_pdf/001.pdf
저장 완료: split_pdf/002.pdf
저장 완료: split_pdf/003.pdf
저장 완료: split_pdf/004.pdf
저장 완료: split_pdf/005.pdf
저장 완료: split_pdf/006.pdf
저장 완료: split_pdf/007.pdf
저장 완료: split_pdf/008.pdf
저장 완료: split_pdf/009.pdf
저장 완료: split_pdf/010.pdf
저장 완료: split_pdf/011.pdf
저장 완료: split_pdf/012.pdf
저장 완료: split_pdf/013.pdf
저장 완료: split_pdf/014.pdf
저장 완료: split_pdf/015.pdf
저장 완료: split_pdf/016.pdf
저장 완료: split_pdf/017.pdf
저장 완료: split_pdf/018.pdf
저장 완료: split_pdf/019.pdf
저장 완료: split_pdf/020.pdf
저장 완료: split_pdf/021.pdf
저장 완료: split_pdf/022.pdf
저장 완료: split_pdf/023.pdf
저장 완료: split_pdf/024.pdf
저장 완료: split_pdf/025.pdf
저장 완료: split_pdf/026.pdf
저장 완료: split_pdf/027.pdf
저장 완료: split_pdf/028.pdf
저장 완료: split_pdf/029.pdf
저장 완료: split_pdf/030.pdf
저장 완료: split_pdf/031.pdf
저장 완료: split_pdf/032.pdf
저장 완료: split_pdf/033.pdf
저장 완료: split_pdf/034.pdf
저장 완료: split_pdf/035.pdf
저장 완료: split_pdf/036.pdf
저장 완료: split_pdf/037.pdf
저장 완료: split_pdf/038.pdf
저장 완료: split_pdf/039.pdf
저장 완료: split_

# 2. 페이지별로 document parser 적용 (markdown 변환)
- PyMuPDF + Upstage 변환

In [8]:
# pip install -qU langchain-core langchain-upstage markdownify

import os
from langchain_upstage import UpstageDocumentParseLoader
from markdownify import markdownify as md

os.environ["UPSTAGE_API_KEY"] = "up_UmN0IgXpQ8eOzwrt2kN386ctcla7C"

In [9]:
import os
import fitz  # PyMuPDF
import json
import tempfile
from langchain_upstage import UpstageDocumentParseLoader
from markdownify import markdownify as md

# --- 1. 이미지 추출 및 제거 기능 (변경사항 없음) ---
def extract_and_remove_images(pdf_path, images_dir):
    """
    PyMuPDF를 사용해 PDF에서 이미지를 추출하여 파일로 저장하고,
    이미지가 제거된 PDF의 메모리상 바이트 데이터를 반환합니다.
    """
    saved_image_filenames = []
    doc = fitz.open(pdf_path)
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]

    for page_index, page in enumerate(doc):
        image_list = page.get_images(full=True)
        for img_index, img in enumerate(image_list):
            xref = img[0]
            try:
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]
                
                img_filename = f"{base_name}_img{img_index}.png"
                img_path = os.path.join(images_dir, img_filename)
                with open(img_path, "wb") as img_file:
                    img_file.write(image_bytes)
                saved_image_filenames.append(img_filename)
                
                img_rect = page.get_image_bbox(img)
                page.add_redact_annot(img_rect, fill=(1, 1, 1))
            
            except Exception as e:
                print(f"⚠️  이미지 처리 중 오류 발생 (파일: {base_name}, 이미지 인덱스: {img_index}): {e}")

        page.apply_redactions()

    image_less_pdf_bytes = doc.write()
    doc.close()
    
    return image_less_pdf_bytes, saved_image_filenames

# --- 2. 마크다운 변환 기능 (변경사항 없음) ---
def convert_cleaned_pdf_to_markdown(pdf_path, ocr="auto"):
    """
    이미지가 제거된 PDF 파일 경로를 받아 Upstage Loader로 마크다운 변환
    """
    loader = UpstageDocumentParseLoader(pdf_path, ocr=ocr)
    pages = loader.load()
    combined_md = "\n\n".join([md(page.page_content) for page in pages if page.page_content.strip()])
    return combined_md

# --- 3. 메인 프로세스 통합 함수 (핵심 수정) ---
def process_pdf_hybrid(pdf_path, output_dir, images_dir, origin_pdf, ocr="auto"):
    """
    단일 PDF에 대해 이미지 추출/제거 및 마크다운 변환을 모두 수행
    """
    print(f"📄 처리 시작: {os.path.basename(pdf_path)}")
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]
    md_path = os.path.join(output_dir, f"{base_name}.md")

    # Step 1: PyMuPDF로 이미지 추출 및 제거
    image_less_pdf_bytes, saved_images = extract_and_remove_images(pdf_path, images_dir)
    if saved_images:
        print(f"    📷 이미지 {len(saved_images)}개 추출 및 저장 완료.")

    # Step 2: Upstage로 텍스트/표를 마크다운으로 변환
    markdown_content = ""
    with tempfile.NamedTemporaryFile(suffix=".pdf", delete=True) as temp_pdf:
        temp_pdf.write(image_less_pdf_bytes)
        temp_pdf.flush()
        markdown_content = convert_cleaned_pdf_to_markdown(temp_pdf.name, ocr=ocr)
    
    print("    ✍️  마크다운 변환 완료.")

    # 💡 변경점: 이미지 링크 추가 로직을 제거하고, 순수 마크다운 텍스트만 저장합니다.
    with open(md_path, "w", encoding="utf-8") as f:
        f.write(markdown_content)
    
    # Step 3: 최종 메타데이터 생성 (이미지 정보는 여기에만 저장)
    page_num = int(base_name) if base_name.isdigit() else 0
    metadata = {
        "source": f"{base_name}.md",
        "origin_pdf": origin_pdf,
        "page_num": page_num,
        "images": saved_images # 추출된 이미지 파일명 리스트
    }
    
    print(f"✅ 처리 완료: {md_path}")
    return metadata

# --- 4. 일괄 처리 함수 (변경사항 없음) ---
def batch_process_all_pdfs(split_dir, output_dir, images_dir, origin_pdf, ocr=False):
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(images_dir, exist_ok=True)
    
    pdf_files = sorted([f for f in os.listdir(split_dir) if f.lower().endswith(".pdf")])
    
    print(f"🚀 총 {len(pdf_files)}개 PDF 파일에 대한 변환을 시작합니다.")
    
    all_metadata = []
    for pdf_file in pdf_files:
        pdf_path = os.path.join(split_dir, pdf_file)
        try:
            metadata = process_pdf_hybrid(pdf_path, output_dir, images_dir, origin_pdf, ocr=ocr)
            all_metadata.append(metadata)
        except Exception as e:
            print(f"❌ 처리 실패: {pdf_file}, 에러: {e}")
            
    print("\n🎉 모든 작업이 완료되었습니다.")
    return all_metadata

# ===== 실행 영역 =====
if __name__ == "__main__":
    # --- 설정 변수 ---
    INPUT_PDF_DIR = "split_pdf"
    MD_OUTPUT_DIR = "md_pdf" # 원하시는 폴더명으로 변경
    IMAGES_OUTPUT_DIR = "md_images"
    ORIGINAL_PDF_NAME = "무역관정산교육_v6_0923.pdf" 
    
    # --- PDF 변환 실행 ---
    final_metadata = batch_process_all_pdfs(
        split_dir=INPUT_PDF_DIR,
        output_dir=MD_OUTPUT_DIR,
        images_dir=IMAGES_OUTPUT_DIR,
        origin_pdf=ORIGINAL_PDF_NAME,
        ocr=False
    )
    
    # --- 메타데이터 파일 저장 ---
    metadata_path = "all_metadata.json"
    with open(metadata_path, "w", encoding="utf-8") as f:
        json.dump(final_metadata, f, ensure_ascii=False, indent=4)
        
    print(f"✅ 최종 메타데이터가 '{metadata_path}' 파일에 저장되었습니다.")

🚀 총 49개 PDF 파일에 대한 변환을 시작합니다.
📄 처리 시작: 001.pdf
    ✍️  마크다운 변환 완료.
✅ 처리 완료: md_pdf/001.md
📄 처리 시작: 002.pdf
    ✍️  마크다운 변환 완료.
✅ 처리 완료: md_pdf/002.md
📄 처리 시작: 003.pdf
    📷 이미지 1개 추출 및 저장 완료.
    ✍️  마크다운 변환 완료.
✅ 처리 완료: md_pdf/003.md
📄 처리 시작: 004.pdf
    📷 이미지 1개 추출 및 저장 완료.
    ✍️  마크다운 변환 완료.
✅ 처리 완료: md_pdf/004.md
📄 처리 시작: 005.pdf
    📷 이미지 1개 추출 및 저장 완료.
    ✍️  마크다운 변환 완료.
✅ 처리 완료: md_pdf/005.md
📄 처리 시작: 006.pdf
    📷 이미지 1개 추출 및 저장 완료.
    ✍️  마크다운 변환 완료.
✅ 처리 완료: md_pdf/006.md
📄 처리 시작: 007.pdf
    ✍️  마크다운 변환 완료.
✅ 처리 완료: md_pdf/007.md
📄 처리 시작: 008.pdf
    📷 이미지 1개 추출 및 저장 완료.
    ✍️  마크다운 변환 완료.
✅ 처리 완료: md_pdf/008.md
📄 처리 시작: 009.pdf
    ✍️  마크다운 변환 완료.
✅ 처리 완료: md_pdf/009.md
📄 처리 시작: 010.pdf
    📷 이미지 1개 추출 및 저장 완료.
    ✍️  마크다운 변환 완료.
✅ 처리 완료: md_pdf/010.md
📄 처리 시작: 011.pdf
    📷 이미지 1개 추출 및 저장 완료.
    ✍️  마크다운 변환 완료.
✅ 처리 완료: md_pdf/011.md
📄 처리 시작: 012.pdf
    📷 이미지 1개 추출 및 저장 완료.
    ✍️  마크다운 변환 완료.
✅ 처리 완료: md_pdf/012.md
📄 처리 시작: 013.pdf
    📷 이미지 1개 추출 및 저장 완료.
    ✍️  

- 메타데이터 확인

In [10]:
import json

# 확인할 파일명
target_filename = "003.md"
metadata_filepath = "all_metadata.json"

# JSON 파일 열기
with open(metadata_filepath, "r", encoding="utf-8") as f:
    all_metadata = json.load(f)

# 데이터 찾기
found_metadata = None
for metadata in all_metadata:
    if metadata.get("source") == target_filename:
        found_metadata = metadata
        break # 찾았으면 반복 중단

# 결과 출력
if found_metadata:
    print(f"✅ [{target_filename}] 파일의 메타데이터를 찾았습니다.")
    # 보기 좋게 출력
    print(json.dumps(found_metadata, indent=4, ensure_ascii=False))
else:
    print(f"❌ [{target_filename}] 파일의 메타데이터를 찾을 수 없습니다.")

✅ [003.md] 파일의 메타데이터를 찾았습니다.
{
    "source": "003.md",
    "origin_pdf": "무역관정산교육_v6_0923.pdf",
    "page_num": 3,
    "images": [
        "003_img0.png"
    ]
}


In [ ]:
# md 파일 내에 이미지도 보여주게 하는 코드
'''
import os
import fitz  # PyMuPDF
import json
import tempfile
from langchain_upstage import UpstageDocumentParseLoader
from markdownify import markdownify as md

# --- 1. PyMuPDF를 사용한 핵심 기능: 이미지 추출 및 제거 ---
# (이 함수는 변경사항 없음)
def extract_and_remove_images(pdf_path, images_dir):
    """
    PyMuPDF를 사용해 PDF에서 이미지를 추출하여 파일로 저장하고,
    이미지가 제거된 PDF의 메모리상 바이트 데이터를 반환합니다.
    """
    saved_image_filenames = []
    doc = fitz.open(pdf_path)
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]

    for page_index, page in enumerate(doc):
        image_list = page.get_images(full=True)
        for img_index, img in enumerate(image_list):
            xref = img[0]
            try:
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]
                
                img_filename = f"{base_name}_p{page_index+1}_img{img_index}.png"
                img_path = os.path.join(images_dir, img_filename)
                with open(img_path, "wb") as img_file:
                    img_file.write(image_bytes)
                saved_image_filenames.append(img_filename)
                
                img_rect = page.get_image_bbox(img)
                page.add_redact_annot(img_rect, fill=(1, 1, 1))
            
            except Exception as e:
                print(f"⚠️  이미지 처리 중 오류 발생 (파일: {base_name}, 이미지 인덱스: {img_index}): {e}")

        page.apply_redactions()

    image_less_pdf_bytes = doc.write()
    doc.close()
    
    return image_less_pdf_bytes, saved_image_filenames


# --- 2. Upstage를 사용한 핵심 기능: 마크다운 변환 ---
# (이 함수는 이제 파일 경로를 받도록 수정)
def convert_cleaned_pdf_to_markdown(pdf_path, ocr=False): # 💡 pdf_bytes 대신 pdf_path를 받음
    """
    이미지가 제거된 PDF 파일 경로를 받아 Upstage Loader로 마크다운 변환
    """
    loader = UpstageDocumentParseLoader(pdf_path, ocr=ocr) # 💡 경로 전달
    pages = loader.load()
    combined_md = "\n\n".join([md(page.page_content) for page in pages if page.page_content.strip()])
    return combined_md


# --- 3. 전체 프로세스를 통합하는 메인 함수 ---
# (임시 파일을 생성하고 경로를 전달하도록 수정)
def process_pdf_hybrid(pdf_path, output_dir, images_dir, origin_pdf, ocr=False):
    """
    단일 PDF에 대해 이미지 추출/제거 및 마크다운 변환을 모두 수행
    """
    print(f"📄 처리 시작: {os.path.basename(pdf_path)}")
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]
    md_path = os.path.join(output_dir, f"{base_name}.md")

    # Step 1: PyMuPDF로 이미지 추출 및 제거 -> 결과는 바이트 데이터
    image_less_pdf_bytes, saved_images = extract_and_remove_images(pdf_path, images_dir)
    if saved_images:
        print(f"    📷 이미지 {len(saved_images)}개 추출 및 저장 완료.")

    # 💡 BUG FIX: 바이트 데이터를 임시 파일에 쓰고, 그 파일의 경로를 사용
    markdown_content = ""
    with tempfile.NamedTemporaryFile(suffix=".pdf", delete=True) as temp_pdf:
        temp_pdf.write(image_less_pdf_bytes)
        temp_pdf.flush() # 모든 데이터를 파일에 확실히 쓰도록 보장

        # Step 2: Upstage로 텍스트/표를 마크다운으로 변환 (이제 파일 경로 사용)
        markdown_content = convert_cleaned_pdf_to_markdown(temp_pdf.name, ocr=ocr)
    
    print("    ✍️  마크다운 변환 완료.")

    # Step 3: 마크다운에 이미지 링크 추가 및 최종 파일 저장
    if saved_images:
        image_md_links = f"\n\n---\n\n## 📷 추출된 이미지\n\n"
        for img_filename in saved_images:
            relative_img_path = f"../{os.path.basename(images_dir)}/{img_filename}"
            image_md_links += f"![{img_filename}]({relative_img_path})\n"
        markdown_content += image_md_links
    
    with open(md_path, "w", encoding="utf-8") as f:
        f.write(markdown_content)
    
    # Step 4: 최종 메타데이터 생성
    page_num = int(base_name) if base_name.isdigit() else 0
    metadata = {
        "source": f"{base_name}.md",
        "origin_pdf": origin_pdf,
        "page_num": page_num,
        "images": saved_images
    }
    
    print(f"✅ 처리 완료: {md_path}")
    return metadata

# --- 4. 여러 파일을 일괄 처리하는 배치 함수 ---
# (이 함수는 변경사항 없음)
def batch_process_all_pdfs(split_dir, output_dir, images_dir, origin_pdf, ocr=False):
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(images_dir, exist_ok=True)
    
    pdf_files = sorted([f for f in os.listdir(split_dir) if f.lower().endswith(".pdf")])
    
    print(f"🚀 총 {len(pdf_files)}개 PDF 파일에 대한 하이브리드 변환을 시작합니다.")
    print(f" - 마크다운 저장 폴더: {output_dir}")
    print(f" - 이미지 저장 폴더: {images_dir}")
    
    all_metadata = []
    for pdf_file in pdf_files:
        pdf_path = os.path.join(split_dir, pdf_file)
        try:
            metadata = process_pdf_hybrid(pdf_path, output_dir, images_dir, origin_pdf, ocr=ocr)
            all_metadata.append(metadata)
        except Exception as e:
            print(f"❌ 처리 실패: {pdf_file}, 에러: {e}")
            
    print("\n🎉 모든 작업이 완료되었습니다.")
    return all_metadata


# ===== 실행 영역 =====
# (이 부분은 변경사항 없음)
if __name__ == "__main__":
    INPUT_PDF_DIR = "split_pdf"
    MD_OUTPUT_DIR = "md_output"
    IMAGES_OUTPUT_DIR = "md_images"
    ORIGINAL_PDF_NAME = "무역관정산교육_v6_0923.pdf"
    
    final_metadata = batch_process_all_pdfs(
        split_dir=INPUT_PDF_DIR,
        output_dir=MD_OUTPUT_DIR,
        images_dir=IMAGES_OUTPUT_DIR,
        origin_pdf=ORIGINAL_PDF_NAME,
        ocr=False
    )
    
    metadata_path = "all_metadata.json"
    with open(metadata_path, "w", encoding="utf-8") as f:
        json.dump(final_metadata, f, ensure_ascii=False, indent=4)
        
    print(f"✅ 최종 메타데이터가 '{metadata_path}' 파일에 저장되었습니다.")
'''